In [2]:
# -*- coding: utf-8 -*-
"""
Clean Testing Script for nnU-Net FLAIR Model
Author: Tech Store
"""

# ============= 1) Install & Configure nnU-Net =============
!pip install -q nnunetv2

import os
from pathlib import Path
import shutil, nibabel as nib, gzip, json, pandas as pd
from nnunetv2.evaluation.evaluate_predictions import compute_metrics_on_folder2

# --- Environment setup ---
os.environ["nnUNet_raw"] = "/kaggle/input/training-flair-2/nnUNet_raw_data"
os.environ["nnUNet_preprocessed"] = "/kaggle/input/training-flair-2/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/kaggle/input/training-flair-2/nnUNet_results"

DATASET_ID = 2
DATASET_NAME = "BonnFCD"
DATASET_DIR = f"Dataset{DATASET_ID:03d}_{DATASET_NAME}"

print(f"✅ Environment configured for dataset: {DATASET_DIR}")

# --- Define paths ---
SRC = Path("/kaggle/input/organized-bonn-fcd-ii-epilepsy-mri-dataset/bonn_fcd_fixed")
RAW_SRC = Path(f"/kaggle/input/training-flair-2/nnUNet_raw_data/{DATASET_DIR}")
RAW_DST = Path(f"/kaggle/working/nnUNet_raw_data/{DATASET_DIR}")

IMAGES_TS_SRC = RAW_SRC / "imagesTs"
IMAGES_TS_DST = RAW_DST / "imagesTs"
LABELS_TS_DST = RAW_DST / "labelsTs"

IMAGES_TS_DST.mkdir(parents=True, exist_ok=True)
LABELS_TS_DST.mkdir(parents=True, exist_ok=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 4.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.5 MB/s eta 0:00:0000:01
 

In [3]:
# ============= 2) Copy Test Data & Labels =============
# Copy test images
for img in sorted(IMAGES_TS_SRC.glob("*_0000.nii*")):
    dst = IMAGES_TS_DST / img.name
    if not dst.exists():
        shutil.copy(img, dst)

# Copy ROI masks
for img in sorted(IMAGES_TS_DST.glob("*_0000.nii*")):
    stem = img.stem.replace("_0000", "")
    sub_folder = SRC / stem / "anat"
    if not sub_folder.exists():
        continue
    mask_files = list(sub_folder.glob("*_roi.nii*"))
    if not mask_files:
        continue
    mask_src = mask_files[0]
    mask_dst = LABELS_TS_DST / f"{stem}{mask_src.suffix}"
    if not mask_dst.exists():
        shutil.copy(mask_src, mask_dst)

# Fix mislabeled gzip files
for f in LABELS_TS_DST.glob("*.nii.gz"):
    try:
        with open(f, "rb") as fp:
            if fp.read(2) != b'\x1f\x8b':  # Not a real gzip
                new_path = f.with_suffix(".nii")
                shutil.move(str(f), str(new_path))
    except Exception:
        pass


In [4]:
# ============= 3) Verify Data Integrity =============
for lbl in sorted(LABELS_TS_DST.glob("*.nii*")):
    stem = lbl.stem.replace('.nii', '')
    img_path = next(
        (p for p in [IMAGES_TS_DST / f"{stem}_0000.nii", IMAGES_TS_DST / f"{stem}_0000.nii.gz"] if p.exists()),
        None
    )
    if not img_path:
        print(f"⚠️ No image found for {stem}")
        continue
    img = nib.load(str(img_path))
    seg = nib.load(str(lbl))
    if img.shape == seg.shape:
        print(f"✅ {stem}: shape OK")
    else:
        print(f"❌ Shape mismatch for {stem}: {img.shape} vs {seg.shape}")

✅ sub-00004: shape OK
✅ sub-00006: shape OK
✅ sub-00009: shape OK
✅ sub-00020: shape OK
✅ sub-00032: shape OK
✅ sub-00034: shape OK
✅ sub-00043: shape OK
✅ sub-00048: shape OK
✅ sub-00061: shape OK
✅ sub-00062: shape OK
✅ sub-00064: shape OK
✅ sub-00066: shape OK
✅ sub-00074: shape OK
✅ sub-00092: shape OK
✅ sub-00095: shape OK
✅ sub-00099: shape OK
✅ sub-00100: shape OK
✅ sub-00103: shape OK
✅ sub-00107: shape OK
✅ sub-00108: shape OK
✅ sub-00121: shape OK
✅ sub-00125: shape OK
✅ sub-00128: shape OK
✅ sub-00134: shape OK
✅ sub-00136: shape OK
✅ sub-00142: shape OK
✅ sub-00144: shape OK
✅ sub-00145: shape OK


In [5]:
# ============= 4) Run Inference =============
CFG, FOLD, CKPT = "3d_fullres", "all", "checkpoint_best.pth"
INP = Path(os.environ["nnUNet_raw"]) / DATASET_DIR / "imagesTs"
OUT = Path(f"/kaggle/working/preds_D{DATASET_ID:03d}_{CFG}_fold{FOLD}_best")
OUT.mkdir(parents=True, exist_ok=True)

print("🔍 Running inference ...")
!nnUNetv2_predict -i "$INP" -o "$OUT" -d "$(printf '%03d' $DATASET_ID)" -c "$CFG" -f $FOLD -chk "$CKPT"
print(f"✅ Predictions saved to: {OUT}")

# ============= 5) Patch dataset.json for evaluation =============
orig_json = Path(f"/kaggle/input/training-flair-2/nnUNet_raw_data/{DATASET_DIR}/dataset.json")
fixed_json = RAW_DST / "dataset_fixed.json"

with open(orig_json) as f:
    ds = json.load(f)

ds["file_ending"] = ".nii.gz"
ds["numTraining"] = 0
ds["numTest"] = len(list(LABELS_TS_DST.glob("*.nii*")))
ds.pop("training", None)

with open(fixed_json, "w") as f:
    json.dump(ds, f, indent=2)
print(f"✅ Patched dataset.json written → {fixed_json}")



🔍 Running inference ...

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 28 cases in the source folder
I am processing 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 28 cases that I would like to predict

Predicting sub-00004:
perform_everything_on_device: True
100%|███████████████████████████████████████████| 27/27 [00:55<00:00,  2.06s/it]
sending off prediction to background worker for resampling and export
done with sub-00004

Predicting sub-00006:
perform_everything_on_device: True
100%|███████████████████████████████████████████| 27/27 [00:44<00:00,  1.64s/it]
sending off prediction to background work

In [7]:
# ============= 6) Evaluate Predictions =============
print("📊 Evaluating predictions...")

from pathlib import Path
from nnunetv2.evaluation.evaluate_predictions import compute_metrics_on_folder2

LABELS_TS_DST = Path("/kaggle/working/nnUNet_raw_data/Dataset002_BonnFCD/labelsTs")
OUT = Path("/kaggle/working/preds_D002_3d_fullres_foldall_best")
fixed_json = Path("/kaggle/working/nnUNet_raw_data/Dataset002_BonnFCD/dataset_fixed.json")
plans_json = Path("/kaggle/input/training-flair-2/nnUNet_results/Dataset002_BonnFCD/nnUNetTrainer__nnUNetPlans__3d_fullres/plans.json")

# 🔍 Detect actual file endings to match dataset.json
sample = next(LABELS_TS_DST.glob("*.nii*"), None)
if sample:
    ending = ".nii.gz" if sample.suffixes[-2:] == ['.nii', '.gz'] else ".nii"
else:
    raise FileNotFoundError("No files found in labelsTs — please verify your ground-truth folder.")

# 🔧 Update dataset.json dynamically
import json
with open(fixed_json) as f:
    data = json.load(f)
data["file_ending"] = ending
with open(fixed_json, "w") as f:
    json.dump(data, f, indent=2)
print(f"✅ Updated dataset.json file_ending → {ending}")

# 🧾 Debug info
print(f"📁 Ground Truth ({LABELS_TS_DST}): {[f.name for f in LABELS_TS_DST.glob(f'*{ending}')][:5]}")
print(f"📁 Predictions ({OUT}): {[f.name for f in OUT.glob('*')] [:5]}")

# 🧮 Run evaluation
try:
    compute_metrics_on_folder2(
        str(LABELS_TS_DST),
        str(OUT),
        str(fixed_json),
        str(plans_json),
        "/kaggle/working/eval_results.json",
        num_processes=1,
    )
    print("✅ Evaluation complete → /kaggle/working/eval_results.json")
except IndexError:
    print("❌ Evaluation failed: No matching files found. Check dataset.json file_ending or filenames.")


📊 Evaluating predictions...
✅ Updated dataset.json file_ending → .nii
📁 Ground Truth (/kaggle/working/nnUNet_raw_data/Dataset002_BonnFCD/labelsTs): ['sub-00095.nii', 'sub-00128.nii', 'sub-00144.nii', 'sub-00100.nii', 'sub-00121.nii']
📁 Predictions (/kaggle/working/preds_D002_3d_fullres_foldall_best): ['sub-00095.nii', 'sub-00128.nii', 'sub-00144.nii', 'sub-00100.nii', 'sub-00121.nii']
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> reader/writer
✅ Evaluation complete → /kaggle/working/eval_results.json


In [16]:
# from pathlib import Path

# labels = Path("/kaggle/working/nnUNet_raw_data/Dataset002_BonnFCD/labelsTs")
# preds  = Path("/kaggle/working/preds_D002_3d_fullres_foldall_best")

# print("Ground truth files:")
# for f in sorted(labels.glob("*.nii*"))[:28]:
#     print(f.name, "→", round(f.stat().st_size/1024/1024,2), "MB")

# print("\nPrediction files:")
# for f in sorted(preds.glob("*.nii*"))[:28]:
#     print(f.name, "→", round(f.stat().st_size/1024/1024,2), "MB")


In [11]:
# 🧩 Inspect eval_results.json structure

import json
from pathlib import Path

res_path = Path("/kaggle/working/eval_results.json")

with open(res_path) as f:
    data = json.load(f)

print("Top-level keys:", list(data.keys()))

# print nested keys if available
for k, v in data.items():
    if isinstance(v, dict):
        print(f"\n🔹 {k}: {list(v.keys())[:10]}")


Top-level keys: ['foreground_mean', 'mean', 'metric_per_case']

🔹 foreground_mean: ['Dice', 'FN', 'FP', 'IoU', 'TN', 'TP', 'n_pred', 'n_ref']

🔹 mean: ['1']


In [12]:
# ============= 7) Show Summary =============
import json, pandas as pd
from pathlib import Path

res_path = Path("/kaggle/working/eval_results.json")

with open(res_path) as f:
    data = json.load(f)

# --- Overall summary ---
print("📈 Overall mean metrics:")
for k, v in data["foreground_mean"].items():
    print(f"  {k:>10}: {v:.4f}")

# --- Per-case metrics ---
if "metric_per_case" in data and isinstance(data["metric_per_case"], dict):
    df = pd.DataFrame.from_dict(data["metric_per_case"], orient="index")
    df = df.round(4)
    display(df)
    print("\n📊 Mean Dice across all cases:", df["Dice"].mean().round(4))
else:
    print("⚠️ No per-case metrics available.")

📈 Overall mean metrics:
        Dice: 0.2209
          FN: 1651.7857
          FP: 1241.7500
         IoU: 0.1562
          TN: 10219685.1429
          TP: 689.8929
      n_pred: 1931.6429
       n_ref: 2341.6786
⚠️ No per-case metrics available.


In [17]:
# ============= 8) Zip for download =============
zip_path = "/kaggle/working/flair_predictions"
shutil.make_archive(zip_path, "zip", OUT)
print(f"📦 Zipped results → {zip_path}.zip")

📦 Zipped results → /kaggle/working/flair_predictions.zip
